<a href="https://colab.research.google.com/github/Arshapjoy/Movie_Recommendation_System_Project/blob/main/notebooks/3_rec_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np


In [21]:

combined_df=pd.read_csv("/content/drive/MyDrive/MCA_project/Final/combined_cleaned.csv")

In [23]:
combined_df.head()

,id,title,audienceScore,tomatoMeter,rating,ratingContents,runtimeMinutes,originalLanguage,director,writer,distributor,cleaned_genres,reviewId,originalScore,reviewText,scoreSentiment,originalScore_normalized
0,space-zombie-bingo,Space Zombie Bingo!,50.000000,65.764818,Unknown,Unknown,75.0,English,George Ormrod,"George Ormrod,John Sabotta",Unknown,"['comedy', 'horror', 'scifi']",NaN,NaN,NaN,NaN,NaN
1,the_green_grass,The Green Grass,55.669356,65.764818,Unknown,Unknown,114.0,English,Tiffany Edwards,Tiffany Edwards,Unknown,['drama'],NaN,NaN,NaN,NaN,NaN
2,love_lies,"Love, Lies",43.000000,65.764818,Unknown,Unknown,120.0,Korean,"Park Heung-Sik,Heung-Sik Park","Ha Young-Joon,Jeon Yun-su,Song Hye-jin",Unknown,['drama'],2739073.0,4/5,Though let down by its routine love triangle n...,POSITIVE,8.0
3,love_lies,"Love, Lies",43.000000,65.764818,Unknown,Unknown,120.0,Korean,"Park Heung-Sik,Heung-Sik Park","Ha Young-Joon,Jeon Yun-su,Song Hye-jin",Unknown,['drama'],2333658.0,3.5/5,"While not perfect, Love, Lies is a worthy disc...",POSITIVE,7.0
4,the_sore_losers_1997,Sore Losers,60.000000,65.764818,Unknown,Unknown,90.0,English,John Michael McCarthy,John Michael McCarthy,Unknown,"['action', 'mysteryandthriller']",NaN,NaN,NaN,NaN,NaN


In [25]:
combined_df["cleaned_genres"].unique()

array(["['comedy', 'horror', 'scifi']", "['drama']",
       "['action', 'mysteryandthriller']", ...,
       "['action', 'comedy', 'fantasy', 'horror', 'scifi']",
       "['animation', 'fantasy', 'horror', 'mysteryandthriller']", nan],
      dtype=object)

In [ ]:
combined_df['genre'] = combined_df['genre'].astype(str)

In [ ]:
def make_genres_list(genre_string):
    if isinstance(genre_string, float):
        genre_string = str(genre_string)
    # ... rest of the function
    genre_list = genre_string.split(', ')
    standardized_genres = []

    for genre in genre_list:
        if genre == 'Science Fiction':
            standardized_genres.append('Sci-Fi')
        else:
            standardized_genres.append(genre)

    if not standardized_genres:
        return np.nan
    else:
        return ' '.join(standardized_genres)


In [ ]:

# Check for non-string values in the 'genre' column
non_string_genres = combined_df['genre'].apply(lambda x: not isinstance(x, str))



In [ ]:
# Convert non-string values to NaN
combined_df['genre'].loc[non_string_genres] = np.nan



In [ ]:
# Apply the make_genres_list function to the 'genre' column
combined_df['genres_list'] = combined_df['genre'].map(make_genres_list)


In [ ]:
combined_df.head()